# Puvodni program - Makro s 2x letaky, predni a zadni strana

In [ ]:
pip install python-pptx

In [ ]:
import shutil
import re
import requests
import bs4
from pptx import Presentation
from pptx.util import Inches, Pt, Cm
#from pptx.enum.text import PP_ALIGN, MSO_ANCHOR, MSO_THEME_COLOR
from pptx.enum.text import *
from pptx.enum.dml import *
import os
import time

#CESTA_SLOZKA = "\Users\Dan\Programy\python\LA_CZ"
#CESTA_TEMPLATE = "\Users\Dan\Programy\python\LA_CZ\MDLZ_1slide.pptx"

start_time = time.time()
#vytvorime prezentaci
try:
  prezka = Presentation("MDLZ_1slide.pptx")
except:
  print("chybí template prezentace")
#pridame uvodni slide a text boxy slide
slide1_register = prezka.slide_layouts[17]
slide1 = prezka.slides.add_slide(slide1_register)
slide1.shapes.title.text = "Týdenní monitoring LA"
slide1.shapes.title.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
# nefunguje dát nadpis do středu: slide1.shapes.title.text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE

height = Cm(1.5)
width = Cm(8)
top1 = Cm(7)
left = Cm(12)
font = "MDLZ BITE TYPE (Nadpisy)"
txBox1 = slide1.shapes.add_textbox(left, top1, width, height)
tf1 = txBox1.text_frame
tf1.text = "zadej číslo týdne"
tf1.paragraphs[0].font.name = font
tf1.paragraphs[0].font.size = Pt(16)
tf1.paragraphs[0].font.color.theme_color = MSO_THEME_COLOR.ACCENT_1

top2 = Cm(10)
txBox2 = slide1.shapes.add_textbox(left, top2, width, height)
tf2 = txBox2.text_frame
tf2.text = "zadej datum"
tf2.paragraphs[0].font.name = font
tf2.paragraphs[0].font.size = Pt(16)
tf2.paragraphs[0].font.color.theme_color = MSO_THEME_COLOR.ACCENT_1

print("uvodni slide vytvoren")

def slide_zakaznika(jmeno_zakaznika):
  """funkce vytvari nový slide s nazvem zakaznika a jeho logem"""
  # vytvorime slide zakaznika a pridam logo
  slide_register = prezka.slide_layouts[17]
  global slide
  slide = prezka.slides.add_slide(slide_register)
  slide.shapes.title.text = ZAKAZNICI[jmeno_zakaznika]
  slide.shapes.title.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
  txBox_plat = slide.shapes.add_textbox(Cm(27), Cm(0.75), Cm(6), Cm(1))
  tf_plat = txBox_plat.text_frame
  tf_plat.text = "LA " + platnost
  tf_plat.paragraphs[0].font.name = "MDLZ BITE TYPE (Nadpisy)"
  tf_plat.paragraphs[0].font.size = Pt(16)
  tf_plat.paragraphs[0].font.color.theme_color = MSO_THEME_COLOR.ACCENT_1
  #pridavame logo
  try:
    logo_cesta = jmeno_zakaznika+"_logo.jpg"
    logo = slide.shapes.add_picture(logo_cesta, left=Cm(0.5), top=Cm(0.5))
  except:
    print("Chybí logo pro", ZAKAZNICI[jmeno_zakaznika])

#### vytvorime druhy slide s nazvem zákazníka
ZAKAZNICI = { "albert-hypermarket":"Albert hypermarket", "albert":"Albert supermarket", "terno":"COOP", "hruska":"Hruška", "kaufland":"Kaufland", "lidl":"Lidl", "makro":"Makro", "penny":"Penny", "billa":"Billa", "globus":"Globus", "tesco":"Tesco" }
ZNACKY = ["milka", "figaro", "opavia", "zlate", "bebe", "piskoty", "oreo", "tuc", "fidorka", "minonky", "tatranky", "kolonada", "brumik", "halls", "3bit", "siesta" ]
URL = "https://letaky.najdislevu.cz/"
HEADERS = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
# udelame for cyklus, který bude dělat slidy pro jednotlivé zákazníky
for zakaznik in ZAKAZNICI.keys():
  pocet_stran = list()
  LA_odkazy = list()
  # pro každého zákazníka najde stranku s letákem a najde odkaz na tento letak a pocet stran
  print("prohlizim LA zakaznika:", zakaznik)
  odkaz = (URL + zakaznik + "/") # stranka s letakem
  html_zakaznik = requests.get(odkaz, HEADERS)
  soup_zakaznik = bs4.BeautifulSoup(html_zakaznik.text, "html.parser")
  try:
    LA_odkazy.append(soup_zakaznik.select_one("#letak1 .oneSites").a["href"]) #najdi prvni letak daneho zakaznika
    pocet_stran.append(int(soup_zakaznik.select_one("#letak1 .infoSite").span.text)) # zjisti pocet stran letaku. dulezite pro omezeni while cyklu a prochazeni letaku
    platnost = soup_zakaznik.select_one("#letak1 .expire").span.text
    if zakaznik == "makro":
      LA_odkazy.append(soup_zakaznik.select_one("#letak2 .oneSites").a["href"])
      pocet_stran.append(int(soup_zakaznik.select_one("#letak2 .infoSite").span.text))
  except:
    pocet_stran = 0
    print(f"{zakaznik} nema platny letak")
  print("pripojim se na tento leták:", LA_odkazy)
  # priklad odkazu na LA_odkazy: https://letaky.najdislevu.cz/albert-hypermarket/letaksh/33948/1/
  # nasledne projde jednotlive strany letaku a hleda nazvy znacek a vypise odkazy do listu na jednotlive strany slevy na letak
  dict_produktu = dict()
  for poradi_odkazu in range(len(LA_odkazy)): 
    c_strany = 1
    while c_strany <= pocet_stran[poradi_odkazu]:
      strany_LA = LA_odkazy[poradi_odkazu].replace(LA_odkazy[poradi_odkazu][-3:], "/"+str(c_strany)+"/")
      html_LA = requests.get(strany_LA, HEADERS)
      soup_LA = bs4.BeautifulSoup(html_LA.text, "html.parser")
      ### tady by měl být for cyklus na prochazeni znacek
      for znacka in ZNACKY:
        html_produktu = soup_LA.find_all(href=re.compile(znacka))
        #a dalsi for cyklus na upravu odkazu, protoze find_all vraci list
        for element in html_produktu:
          if c_strany == pocet_stran[poradi_odkazu]:
            dict_produktu[element["href"]] = "posledni strana"
          else:
            dict_produktu[element["href"]] = c_strany
          # vystup ze slovniku: https://akce.najdislevu.cz/1946423-oreo-susenky.html : cislo strany
      c_strany += 1 #změni cislo stranky

  print("toto jsou odkazy na produkty v akci:", dict_produktu.keys())
  # vytvorime slide zakaznika a pridam logo
  slide_zakaznika(zakaznik)
  print("vytvoril jsem slide pro", zakaznik)

  #projdu v ramci zakaznika jednotlivé odkazy na produkty v letaky a vytáhnut odkazy na obrázky a rovnou pridám do prezky
  #musím rovnou i odchytat informaci, pokud je položka na 1. straně LA
  c_obrazku = 1
  
  for produkt in dict_produktu.keys():
    print("pridavam obrazek c.", c_obrazku)
    if c_obrazku%5 == 0:
      slide_zakaznika(zakaznik)

    html_detail_prod = requests.get(produkt, HEADERS)
    soup_detail_prod = bs4.BeautifulSoup(html_detail_prod.text, "html.parser")
    odkaz_pic = soup_detail_prod.find(id="acImgDet")["src"] #ziskavam odkaz na obrazek ve stringu
    #umisteni = soup_detail_prod.find(string=re.compile(". strana")) #ziskavam umisteni/cislo stranky letaku daneho produktu
    html_obr = requests.get(odkaz_pic, HEADERS, stream = True)
    jmeno_obr = zakaznik+".jpg"
    with open(jmeno_obr,"wb") as obr:
        shutil.copyfileobj(html_obr.raw, obr)
    #cesta_LA = os.path.join("C:", os.sep, os.getcwd() + "\\" + jmeno_obr)
    #pridame obrazek letaku do prezentace, pokud je obrazek na 1. strane, pridame i pole s umistenim
    if c_obrazku%4 == 1: #1. obrazek umistime vlevo nahoru
      left=Cm(3) 
      top=Cm(4)
      height=Cm(6.5)
      width=Cm(6.5)
      letak = slide.shapes.add_picture(jmeno_obr, left, top, height, width)
      if dict_produktu[produkt] == 1:
        txBox1 = slide.shapes.add_textbox(left, top-Cm(1), width=Cm(2), height=Cm(1))
        tf1 = txBox1.text_frame
        tf1.text = "Přední strana"
      if dict_produktu[produkt] == "posledni strana":
        txBox1 = slide.shapes.add_textbox(left, top-Cm(1), width=Cm(2), height=Cm(1))
        tf1 = txBox1.text_frame
        tf1.text = "Poslední strana"
      os.remove(jmeno_obr)
    elif c_obrazku%4 == 2: #2. obrazek umistime vpravo nahoru
      left=Cm(24) 
      top=Cm(4)
      height=Cm(6.5)
      width=Cm(6.5)
      letak = slide.shapes.add_picture(jmeno_obr, left, top, height, width)
      if dict_produktu[produkt] == 1:
        txBox1 = slide.shapes.add_textbox(left, top-Cm(1), width=Cm(2), height=Cm(1))
        tf1 = txBox1.text_frame
        tf1.text = "Přední strana"
      if dict_produktu[produkt] == "posledni strana":
        txBox1 = slide.shapes.add_textbox(left, top-Cm(1), width=Cm(2), height=Cm(1))
        tf1 = txBox1.text_frame
        tf1.text = "Poslední strana"
      os.remove(jmeno_obr)
    elif c_obrazku%4 == 3: #3. obrazek umistime vlevo dolu
      left=Cm(3) 
      top=Cm(12)
      height=Cm(6.5)
      width=Cm(6.5)
      letak = slide.shapes.add_picture(jmeno_obr, left, top, height, width)
      if dict_produktu[produkt] == 1:
        txBox1 = slide.shapes.add_textbox(left, top-Cm(1), width=Cm(2), height=Cm(1))
        tf1 = txBox1.text_frame
        tf1.text = "Přední strana"
      if dict_produktu[produkt] == "posledni strana":
        txBox1 = slide.shapes.add_textbox(left, top-Cm(1), width=Cm(2), height=Cm(1))
        tf1 = txBox1.text_frame
        tf1.text = "Poslední strana"
      os.remove(jmeno_obr)
    elif c_obrazku%4 == 0: #4. obrazek umistime vpravo dolu
      left=Cm(24) 
      top=Cm(12)
      height=Cm(6.5)
      width=Cm(6.5)
      letak = slide.shapes.add_picture(jmeno_obr, left, top, height, width)
      if dict_produktu[produkt] == 1:
        txBox1 = slide.shapes.add_textbox(left, top-Cm(1), width=Cm(2), height=Cm(1))
        tf1 = txBox1.text_frame
        tf1.text = "Přední strana"
      if dict_produktu[produkt] == "posledni strana":
        txBox1 = slide.shapes.add_textbox(left, top-Cm(1), width=Cm(2), height=Cm(1))
        tf1 = txBox1.text_frame
        tf1.text = "Poslední strana"
      os.remove(jmeno_obr)
    # pridam do poradi do cisla obrazku, ktery budu následne opět přidávat
    c_obrazku += 1

end_time = time.time()
working_time = end_time - start_time
print(f"Ukladam prezentaci. Pracoval jsem {time.localtime(working_time).tm_hour} hodin, {time.localtime(working_time).tm_min} minut a {time.localtime(working_time).tm_sec} sekund.")
prezka.save("LA monitoring.pptx")


# vyreseni Makra a 2 letaky - FINAL

In [2]:
pip install python-pptx

     |████████████████████████████████| 10.1 MB 5.1 MB/s 
     |████████████████████████████████| 149 kB 49.1 MB/s 
  Created wheel for python-pptx: filename=python_pptx-0.6.21-py3-none-any.whl size=470951 sha256=97067bd9087e79eb1eabd9e3cd4d3495247b3a997f341d6211e14df20684eca5
  Stored in directory: /root/.cache/pip/wheels/a7/ab/f4/52560d0d4bd4055e9261c6df6e51c7b56c2b23cca3dee811a3
Successfully built python-pptx


In [4]:
import shutil
import re
import requests
import bs4
from pptx import Presentation
from pptx.util import Inches, Pt, Cm
#from pptx.enum.text import PP_ALIGN, MSO_ANCHOR, MSO_THEME_COLOR
from pptx.enum.text import *
from pptx.enum.dml import *
import os
import time

#CESTA_SLOZKA = "\Users\Dan\Programy\python\LA_CZ"
#CESTA_TEMPLATE = "\Users\Dan\Programy\python\LA_CZ\MDLZ_1slide.pptx"

start_time = time.time()
#vytvorime prezentaci
try:
  prezka = Presentation("MDLZ_1slide.pptx")
except:
  print("chybí template prezentace")
#pridame uvodni slide a text boxy slide
slide1_register = prezka.slide_layouts[17]
slide1 = prezka.slides.add_slide(slide1_register)
slide1.shapes.title.text = "Týdenní monitoring LA"
slide1.shapes.title.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
# nefunguje dát nadpis do středu: slide1.shapes.title.text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE

height = Cm(1.5)
width = Cm(8)
top1 = Cm(7)
left = Cm(12)
font = "MDLZ BITE TYPE (Nadpisy)"
txBox1 = slide1.shapes.add_textbox(left, top1, width, height)
tf1 = txBox1.text_frame
tf1.text = "zadej číslo týdne"
tf1.paragraphs[0].font.name = font
tf1.paragraphs[0].font.size = Pt(16)
tf1.paragraphs[0].font.color.theme_color = MSO_THEME_COLOR.ACCENT_1

top2 = Cm(10)
txBox2 = slide1.shapes.add_textbox(left, top2, width, height)
tf2 = txBox2.text_frame
tf2.text = "zadej datum"
tf2.paragraphs[0].font.name = font
tf2.paragraphs[0].font.size = Pt(16)
tf2.paragraphs[0].font.color.theme_color = MSO_THEME_COLOR.ACCENT_1

print("uvodni slide vytvoren")

def slide_zakaznika(jmeno_zakaznika):
  """funkce vytvari nový slide s nazvem zakaznika a jeho logem"""
  # vytvorime slide zakaznika a pridam logo
  slide_register = prezka.slide_layouts[17]
  global slide
  slide = prezka.slides.add_slide(slide_register)
  slide.shapes.title.text = ZAKAZNICI[jmeno_zakaznika]
  slide.shapes.title.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
  txBox_plat = slide.shapes.add_textbox(Cm(27), Cm(0.75), Cm(6), Cm(1))
  tf_plat = txBox_plat.text_frame
  tf_plat.text = "LA " + platnost
  tf_plat.paragraphs[0].font.name = "MDLZ BITE TYPE (Nadpisy)"
  tf_plat.paragraphs[0].font.size = Pt(16)
  tf_plat.paragraphs[0].font.color.theme_color = MSO_THEME_COLOR.ACCENT_1
  #pridavame logo
  try:
    logo_cesta = jmeno_zakaznika+"_logo.jpg"
    logo = slide.shapes.add_picture(logo_cesta, left=Cm(0.5), top=Cm(0.5), width=Cm(4), height=Cm(2))
  except:
    print("Chybí logo pro", ZAKAZNICI[jmeno_zakaznika])


#### vytvorime druhy slide s nazvem zákazníka
ZAKAZNICI = { "albert-hypermarket":"Albert hypermarket", "albert":"Albert supermarket", "terno":"COOP", "hruska":"Hruška", "kaufland":"Kaufland", "lidl":"Lidl", "makro":"Makro", "penny":"Penny", "billa":"Billa", "globus":"Globus", "tesco":"Tesco" }
ZNACKY = ["milka", "figaro", "opavia", "zlate", "bebe", "piskoty", "oreo", "tuc", "fidorka", "minonky", "tatranky", "kolonada", "brumik", "halls", "3bit", "siesta", "biscuits" ]
URL = "https://letaky.najdislevu.cz/"
HEADERS = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
# udelame for cyklus, který bude dělat slidy pro jednotlivé zákazníky
for zakaznik in ZAKAZNICI.keys():
  pocet_stran = list()
  LA_odkazy = list()
  # pro každého zákazníka najde stranku s letákem a najde odkaz na tento letak a pocet stran
  print("prohlizim LA zakaznika:", zakaznik)
  odkaz = (URL + zakaznik + "/") # stranka s letakem
  html_zakaznik = requests.get(odkaz, HEADERS)
  soup_zakaznik = bs4.BeautifulSoup(html_zakaznik.text, "html.parser")
  try:
    LA_odkazy.append(soup_zakaznik.select_one("#letak1 .oneSites").a["href"]) #najdi prvni letak daneho zakaznika
    pocet_stran.append(int(soup_zakaznik.select_one("#letak1 .infoSite").span.text)) # zjisti pocet stran letaku. dulezite pro omezeni while cyklu a prochazeni letaku
    platnost = soup_zakaznik.select_one("#letak1 .expire").span.text
    if zakaznik == "makro":
      LA_odkazy.append(soup_zakaznik.select_one("#letak2 .oneSites").a["href"])
      pocet_stran.append(int(soup_zakaznik.select_one("#letak2 .infoSite").span.text))
  except:
    pocet_stran = 0
    print(f"{zakaznik} nema platny letak")
  print("pripojim se na tento leták:", LA_odkazy)
  # priklad odkazu na LA_odkazy: https://letaky.najdislevu.cz/albert-hypermarket/letaksh/33948/1/
  # nasledne projde jednotlive strany letaku a hleda nazvy znacek a vypise odkazy do listu na jednotlive strany slevy na letak
  dict_produktu = dict()
  for poradi_odkazu in range(len(LA_odkazy)): 
    c_strany = 1
    while c_strany <= pocet_stran[poradi_odkazu]:
      strany_LA = LA_odkazy[poradi_odkazu].replace(LA_odkazy[poradi_odkazu][-3:], "/"+str(c_strany)+"/")
      html_LA = requests.get(strany_LA, HEADERS)
      soup_LA = bs4.BeautifulSoup(html_LA.text, "html.parser")
      ### tady by měl být for cyklus na prochazeni znacek
      for znacka in ZNACKY:
        html_produktu = soup_LA.find_all(href=re.compile(znacka))
        #a dalsi for cyklus na upravu odkazu, protoze find_all vraci list
        for element in html_produktu:
          if c_strany == pocet_stran[poradi_odkazu]:
            dict_produktu[element["href"]] = "posledni strana"
          else:
            dict_produktu[element["href"]] = c_strany
          # vystup ze slovniku: https://akce.najdislevu.cz/1946423-oreo-susenky.html : cislo strany
      c_strany += 1 #změni cislo stranky

  print("toto jsou odkazy na produkty v akci:", dict_produktu.keys())
  # vytvorime slide zakaznika a pridam logo
  slide_zakaznika(zakaznik)
  print("vytvoril jsem slide pro", zakaznik)

  #projdu v ramci zakaznika jednotlivé odkazy na produkty v letaky a vytáhnut odkazy na obrázky a rovnou pridám do prezky
  #musím rovnou i odchytat informaci, pokud je položka na 1. straně LA
  c_obrazku = 1
  
  for produkt in dict_produktu.keys():
    print("pridavam obrazek c.", c_obrazku)
    if c_obrazku%4 == 1:
      slide_zakaznika(zakaznik)

    html_detail_prod = requests.get(produkt, HEADERS)
    soup_detail_prod = bs4.BeautifulSoup(html_detail_prod.text, "html.parser")
    odkaz_pic = soup_detail_prod.find(id="acImgDet")["src"] #ziskavam odkaz na obrazek ve stringu
    #umisteni = soup_detail_prod.find(string=re.compile(". strana")) #ziskavam umisteni/cislo stranky letaku daneho produktu
    html_obr = requests.get(odkaz_pic, HEADERS, stream = True)
    jmeno_obr = zakaznik+".jpg"
    with open(jmeno_obr,"wb") as obr:
        shutil.copyfileobj(html_obr.raw, obr)
    #cesta_LA = os.path.join("C:", os.sep, os.getcwd() + "\\" + jmeno_obr)
    #pridame obrazek letaku do prezentace, pokud je obrazek na 1. strane, pridame i pole s umistenim
    if c_obrazku%4 == 1: #1. obrazek umistime vlevo nahoru
      left=Cm(3) 
      top=Cm(4)
      height=Cm(6.5)
      width=Cm(6.5)
      letak = slide.shapes.add_picture(jmeno_obr, left, top, height, width)
      txBox1 = slide.shapes.add_textbox(left, top-Cm(1), width=Cm(2), height=Cm(1))
      tf1 = txBox1.text_frame
      tf1.text = "strana " + str(dict_produktu[produkt])
      os.remove(jmeno_obr)
    elif c_obrazku%4 == 2: #2. obrazek umistime vpravo nahoru
      left=Cm(24) 
      top=Cm(4)
      height=Cm(6.5)
      width=Cm(6.5)
      letak = slide.shapes.add_picture(jmeno_obr, left, top, height, width)
      txBox1 = slide.shapes.add_textbox(left, top-Cm(1), width=Cm(2), height=Cm(1))
      tf1 = txBox1.text_frame
      tf1.text = "strana " + str(dict_produktu[produkt])
      os.remove(jmeno_obr)
    elif c_obrazku%4 == 3: #3. obrazek umistime vlevo dolu
      left=Cm(3) 
      top=Cm(12)
      height=Cm(6.5)
      width=Cm(6.5)
      letak = slide.shapes.add_picture(jmeno_obr, left, top, height, width)
      txBox1 = slide.shapes.add_textbox(left, top-Cm(1), width=Cm(2), height=Cm(1))
      tf1 = txBox1.text_frame
      tf1.text = "strana " + str(dict_produktu[produkt])
      os.remove(jmeno_obr)
    elif c_obrazku%4 == 0: #4. obrazek umistime vpravo dolu
      left=Cm(24) 
      top=Cm(12)
      height=Cm(6.5)
      width=Cm(6.5)
      letak = slide.shapes.add_picture(jmeno_obr, left, top, height, width)
      txBox1 = slide.shapes.add_textbox(left, top-Cm(1), width=Cm(2), height=Cm(1))
      tf1 = txBox1.text_frame
      tf1.text = "strana " + str(dict_produktu[produkt])
      os.remove(jmeno_obr)
    # pridam do poradi do cisla obrazku, ktery budu následne opět přidávat
    c_obrazku += 1

end_time = time.time()
working_time = end_time - start_time
print(f"Ukladam prezentaci. Pracoval jsem {time.localtime(working_time).tm_hour} hodin, {time.localtime(working_time).tm_min} minut a {time.localtime(working_time).tm_sec} sekund.")
prezka.save("LA monitoring.pptx")

#potřeba vyřešit poslední stranu

uvodni slide vytvoren
prohlizim LA zakaznika: albert-hypermarket
pripojim se na tento leták: ['https://letaky.najdislevu.cz/albert-hypermarket/letaksh/34717/1/']
toto jsou odkazy na produkty v akci: {'https://akce.najdislevu.cz/1994140-milka-liskooriskova-pomazanka.html': 2, 'https://akce.najdislevu.cz/1994164-bebe-susenky.html': 3, 'https://akce.najdislevu.cz/1994176-milka-egg-spoon-oreo.html': 4, 'https://akce.najdislevu.cz/1994210-milka-susenky.html': 5, 'https://akce.najdislevu.cz/1994196-oreo-susenky.html': 5, 'https://akce.najdislevu.cz/1994205-kolonada-oplatky.html': 5, 'https://akce.najdislevu.cz/1994429-figaro-cokolada.html': 15}
vytvoril jsem slide pro albert-hypermarket
pridavam obrazek c. 1
pridavam obrazek c. 2
pridavam obrazek c. 3
pridavam obrazek c. 4
pridavam obrazek c. 5
pridavam obrazek c. 6
pridavam obrazek c. 7
prohlizim LA zakaznika: albert
pripojim se na tento leták: ['https://letaky.najdislevu.cz/albert/letaksh/34719/1/']
toto jsou odkazy na produkty v akci: {'h